# **Memory**

## **What's covered?**
1. What is Memory?
2. Building Memory into a system
3. Depricated Classes
4. Buidling End-to-end Conversational AI Bot by designing Memory from Scratch
    - Step 1: Import Chat Model and Configure the API Key
    - Step 2: Create Chat Template
    - Step 3: Create a Output Parser
    - Step 4: Initialize the Memory
    - Step 5: Build a Chain
    - Step 6: Invoke the chain
    - Step 7: Saving to memory
    - Step 8: Run Step 6 and 7 in a loop
5. Saving a Chat History to a Pickle File
6. Loading a Chat History from a Pickle File
7. Manage Message History with SQLChatMessageHistory and Adding Session ID

## **What is Memory?**
Memory is a cognitive function that allows people to store, retrieve, and use information to understand their present and future. Consider the frustration of working with a colleague who forgets everything you tell them, requiring constant repetition! As AI agents undertake more complex tasks involving numerous user interactions, equipping them with memory becomes equally crucial for efficiency and user satisfaction. With memory, agents can learn from feedback and adapt to users' preferences. 

Most LLM applications have a conversational interface. An essential component of a conversation is being able to refer to information introduced earlier in the conversation. At bare minimum, a conversational system should be able to access some window of past messages directly. A more complex system will need to have a world model that it is constantly updating, which allows it to do things like maintain information about entities and their relationships.

We call this ability to store information about past interactions "memory". 

## **Building memory into a system**
The two core design decisions in any memory system are:
- How state is stored
- How state is queried

## **Depricated Classes**
- ConversationBufferMemory
- ConversationStringBufferMemory
- ConversationBufferWindowMemory
- ConversationTokenBufferMemory
- ConversationSummaryMemory
- ConversationSummaryBufferMemory
- VectorStoreRetrieverMemory

## **Buidling End-to-end Conversational AI Bot by designing Memory from Scratch**

### **Steps:**
1. Import Chat Model and Configure the API Key
2. Create Chat Template
3. Create a Output Parser
4. Initialize the Memory
5. Build a Chain
6. Invoke the chain with human_input and chat_history
7. Saving to memory
8. Run Step 6 and 7 in a loop

<img src="images/memory.png">

### **Step 1: Import Chat Model and Configure the API Key**

In [1]:
# Step 1 - Import Chat Model and Configure the API Key

from langchain_openai import ChatOpenAI

# Setup API Key
f = open('keys/.openai_api_key.txt')
OPENAI_API_KEY = f.read()

# Set the OpenAI Key and initialize a ChatModel
chat_model = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4o-mini")

### **Step 2: Create Chat Template**

In [2]:
# Step 2 - Create Chat Template

from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

chat_template = ChatPromptTemplate(
    messages = [
        # The persistent system prompt
        SystemMessage(
            content="You are a chatbot having a conversation with a human."
        ),
        # Creating a chat_history placeholder
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  
        # Human Prompt
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),
    ]
)

### **Step 3: Create a Output Parser**

In [3]:
# Step 3 - Create a Output Parser

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

### **Step 4: Initialize the Memory**

In [4]:
# Step 4 - Initialize the Memory
from langchain_core.runnables import RunnableLambda

memory_buffer = {"history": []}

def get_history_from_buffer(human_input):
    return memory_buffer["history"]

runnable_get_history_from_buffer = RunnableLambda(get_history_from_buffer)

#### **RunnablePassthrough:** RunnablePassthrough on its own allows you to pass inputs unchanged.

### **Step 5: Build a Chain**

In [5]:
# Step 5 - Build a Chain (Another way)
from langchain_core.runnables import RunnablePassthrough

# Define a chain
chain = RunnablePassthrough.assign(
        chat_history=runnable_get_history_from_buffer
        ) | chat_template | chat_model | output_parser

### **Step 6: Invoke the chain**

In [6]:
# Step 6 - Invoke the chain with human_input and chat_history

query = {"human_input": "Hi, How are you?"}

response = chain.invoke(query)

response

"Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"

In [7]:
memory_buffer

{'history': []}

### **Step 7: Saving to memory**

In [8]:
# Step 7 - Saving to memory
from langchain_core.messages import HumanMessage, AIMessage

memory_buffer["history"].append(HumanMessage(content=query["human_input"]))
memory_buffer["history"].append(AIMessage(content=response))

memory_buffer

{'history': [HumanMessage(content='Hi, How are you?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={}, response_metadata={})]}

### **Step 8 - Run Step 6 and 7 in a loop**

In [10]:
# Step 8 - Run Step 6 and 7 in a loop

while True:
    query = {"human_input" : input('Enter your input: ')}
    print(f"*User: {query['human_input']}")
    if query["human_input"] in ['bye', 'quit', 'exit']:
        break
    response = chain.invoke(query)
    print(f"*AI: {response}")

    memory_buffer["history"].append(HumanMessage(content=query["human_input"]))
    memory_buffer["history"].append(AIMessage(content=response))

Enter your input:  My name is Kanav


*User: My name is Kanav
*AI: Hello Kanav! It's nice to meet you. How can I assist you today?


Enter your input:  just exploring


*User: just exploring
*AI: That's great! Feel free to ask me anything or share any topics you'd like to explore. I'm here to help and provide information.


Enter your input:  that's good to know


*User: that's good to know
*AI: I'm glad to hear that! If you have any questions or need assistance, don't hesitate to ask. I'm here to help.


Enter your input:  what;s my name?


*User: what;s my name?
*AI: Your name is Kanav.


Enter your input:  exit


*User: exit


In [11]:
memory_buffer["history"]

[HumanMessage(content='Hi, How are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='My name is Kanav', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello Kanav! It's nice to meet you. How can I assist you today?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='just exploring', additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's great! Feel free to ask me anything or share any topics you'd like to explore. I'm here to help and provide information.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="that's good to know", additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm glad to hear that! If you have any questions or need assistance, don't hesitate to ask. I'm here to help.", 

## **Saving a Chat History to a Pickle File**

**Let's now learn to save this history on the disk so that whenever we can load the history whenever we chat with our assistant.**

In [12]:
import pickle

chat_history = pickle.dumps(memory_buffer)

with open("chats_data/conversation_memory.pkl", "wb") as f:
    f.write(chat_history)

## **Loading a Chat History from a Pickle File**

In [13]:
chat_history_loaded = pickle.load(open("chats_data/conversation_memory.pkl", "rb"))

In [14]:
chat_history_loaded

{'history': [HumanMessage(content='Hi, How are you?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Kanav', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello Kanav! It's nice to meet you. How can I assist you today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='just exploring', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's great! Feel free to ask me anything or share any topics you'd like to explore. I'm here to help and provide information.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="that's good to know", additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm glad to hear that! If you have any questions or need assistance, don't hesitate to ask. I

## **Manage Message History with SQLChatMessageHistory and Adding Session ID**

`ChatMessageHistory` allows us to store separate conversation histories per user or session which is often done by the real-time chatbots. `session_id` is used to distinguish between separate conversations.

In order to use it, we can use a `get_session_history` function which take `session_id` and returns a message history object.

There is a support of many `Memory` components under `langchain_community.chat_message_histories`, like:
1. AstraDBChatMessageHistory
2. DynamoDBChatMessageHistory
3. CassandraChatMessageHistory
4. ElasticsearchChatMessageHistory
5. KafkaChatMessageHistory
6. MongoDBChatMessageHistory
7. RedisChatMessageHistory
8. PostgresChatMessageHistory
9. SQLChatMessageHistory

**[Click Here](https://python.langchain.com/docs/integrations/memory/)** to read more.

### **Usage**

To use the storage you need to provide only 2 things:

1. **Session Id** - a unique identifier of the session, like user name, email, chat id etc.
2. **Connection string**
    - For SQL (SQLAlchemy) - A string that specifies the database connection. It will be passed to SQLAlchemy create_engine function.
    - For SQLite - A string that specifies the database connection. For SQLite, that string is slqlite:/// followed by the name of the database file. If that file doesn't exist, it will be created.

In [11]:
# ! pip install -U langchain-community SQLAlchemy

In [12]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [13]:
# Set the OpenAI Key and initialize a ChatModel

from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [14]:
# Create a connection with the database and 
# return the chat message history for a session id

from langchain_community.chat_message_histories import SQLChatMessageHistory

def get_session_message_history_from_db(session_id):
    chat_message_history = SQLChatMessageHistory(
                                   session_id=session_id, 
                                   connection="sqlite:///chats_data/sqlite.db"
                               )
    return chat_message_history

In [15]:
# Create a chat template

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_template = ChatPromptTemplate(
                messages=[
                    ("system", "You are a helpful AI assistant."), 
                    MessagesPlaceholder(variable_name="history"), 
                    ("human", "{human_input}")
                ]
)

In [16]:
# Defining the chain

chain = chat_template | chat_model | output_parser

In [17]:
# Use RunnableWithMessageHistory to load 
from langchain_core.runnables.history import RunnableWithMessageHistory

conversation_chain = RunnableWithMessageHistory(
                        chain, 
                        get_session_message_history_from_db,
                        input_messages_key="human_input", 
                        history_messages_key="history"
                    )

In [18]:
# This is where we configure the session id
user_id = "thataiguy"
config = {"configurable": {"session_id": user_id}}

input_prompt = {"human_input": "My name is ThatAIGuy. Can you tell me the capital of Himachal?"}
response = conversation_chain.invoke(input_prompt, config=config)

response

'Yes, the capital of Himachal Pradesh is Shimla.'

In [19]:
# This is where we configure the session id
user_id = "kanav"
config = {"configurable": {"session_id": user_id}}

input_prompt = {"human_input": "My name is Kanav Bansal. What is the biggest state in India?"}
response = conversation_chain.invoke(input_prompt, config=config)

response

'The biggest state in India is Rajasthan.'

In [20]:
def chat_bot(session_id, prompt):
    config = {"configurable": {"session_id": user_id}}
    input_prompt = {"human_input": prompt}

    response = conversation_chain.invoke(input_prompt, config=config)

    return response

In [21]:
user_id = "thataiguy"
input_prompt = "Do you remember my name?"
chat_bot(session_id=user_id, prompt=input_prompt)

'Yes, your name is ThatAIGuy.'

In [22]:
user_id = "kanav"
input_prompt = "Do you remember my name?"
chat_bot(session_id=user_id, prompt=input_prompt)

'Yes, your name is Kanav Bansal.'